# Measuring galaxy redshift with cross-correlation algorithm

This notebook attempts to follow the workflow that uses IRAF tasks, described here http://tdc-www.harvard.edu/iraf/rvsao/xcsao/xcsao.proc.html

Observed spectrum from LEGA-C: LEGA-C is a galaxy survey of about 3000 galaxies at z~0.8 and M* > 10^10 M_sun in the COSMOS field. The spectra sample the rest-frame optical between ~3000A and 5000A at high resolution and very high signal-to-noise ratio. More information about the survey can be found here: http://www.mpia.de/home/legac/

Template from Pacifici et al. 2012.

**Developer Notes:**
    - This workflow will be rendered in a few simple clicks in specviz
    - Preparing the template outside the correlation function allows for applications to different science cases

Author: Ivo Busko

In [ ]:
import os
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as pl

from scipy.signal.windows import tukey
import astropy
import astropy.units as u
from astropy.table import QTable
from astropy.nddata import StdDevUncertainty
from astropy.modeling.polynomial import Chebyshev1D
from astropy import constants as const
from astropy.io import fits, ascii

import specutils
from specutils.fitting import continuum, find_lines_threshold, find_lines_derivative
from specutils.spectra.spectrum1d import Spectrum1D
from specutils.manipulation.resample import FluxConservingResampler, \
    SplineInterpolatedResampler, LinearInterpolatedResampler
from specutils.analysis import correlation
from specutils.spectra.spectral_region import SpectralRegion
from specutils.manipulation.extract_spectral_region import extract_region
from specutils.manipulation.utils import linear_exciser
from specutils.manipulation import noise_region_uncertainty
from specutils.manipulation import gaussian_smooth

# Check versions
print("Numpy: ",np.__version__)
print("Astropy: ",astropy.__version__)
print("Specutils: ",specutils.__version__)
print("")
print("They should be:")
print("Numpy:  1.18.1")
print("Astropy:  4.0")
print("Specutils:  1.0")

### Define data files:

In [ ]:
# Files are on box

# Observation and weight.
file1d = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/redshift_crosscorr/legac_M1_v3.7_spec1d_130902.fits'
file1dwht = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/redshift_crosscorr/legac_M1_v3.7_wht1d_130902.fits'
# Template.
template_file = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/redshift_crosscorr/00006.dat'

# Plot limits
sp_xlim = [3000., 9000.]

### Read observation and template:

In [ ]:
# Observation
hdu1d = fits.open(file1d)
hdu1dwht = fits.open(file1dwht)

flux = hdu1d[0].data
wht = hdu1dwht[0].data
unc = 1./ np.sqrt(wht)
wave = np.arange(flux.shape[0])*hdu1d[0].header['CD1_1'] + hdu1d[0].header['CRVAL1']

spec_unit = u.Unit('10^-19 erg s^-1 cm^-2 angstrom^-1')
dataspec = QTable([wave*u.angstrom, flux*spec_unit, wht, unc*spec_unit], 
                   names=('wavelength','flux','weight','uncertainty'))
dataspec_sub = dataspec[dataspec['weight']>0.]
dataspec_sub

In [ ]:
# Now make it into a Spectrum1D instance.
obs = Spectrum1D(spectral_axis=dataspec_sub['wavelength'], 
                 flux=dataspec_sub['flux'], 
                 uncertainty=StdDevUncertainty(dataspec_sub['uncertainty']))

In [ ]:
# Template
template = ascii.read(template_file)
factor = 2.E-5 * obs.flux.unit # normalize template to a sensible range
template = Spectrum1D(spectral_axis=template['col1']*u.AA, 
                      flux=template['col2']*factor)

In [ ]:
# Note that in this and in subsequent plots, we are showing just the wavelength range of
# interest. The template covers a significantly wider range.
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(obs.wavelength, obs.flux, linewidth=0.5, label='obs')
pl.plot(template.wavelength, template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('Input data')

## Preprocess Spectra

### Subtract continuum

In [ ]:
continuum_model = continuum.fit_generic_continuum(obs) 
p_obs = obs - continuum_model(obs.wavelength)
continuum_model = continuum.fit_generic_continuum(template, model=Chebyshev1D(5)) 
p_template = template - continuum_model(template.wavelength)

In [ ]:
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(p_obs.wavelength, p_obs.flux, linewidth=0.5, label='obs')
pl.plot(p_template.wavelength, p_template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('After continuum subtraction')

### Smooth observation

The IRAF task XCORR works in Fourier space. In there, it applies a cosine bell filter (raised-cosine filter) to the observed spectrum, before multiplying together the two Fourier transforms. Here, we are working in data space, thus we emulate the filter operation by convolving the observed spectrum with a windowed sinc smoothing function.

In [ ]:
# Smooth data with sinc kernel
fc = 0.25  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
b = 0.49   # Transition band, as a fraction of the sampling rate (in (0, 0.5)).

# The IRAF task uses the above values. Here, we try
# a much lower cutoff frequency to really dampen the
# high frequency structure in the observed spectrum.
fc = 0.05 

N = int(np.ceil((4 / b)))
if not N % 2:  # N must be odd
    N += 1
n = np.arange(N)
 
# Compute sinc filter and Blackman window. Multiply filter 
# by window and normalize to get unity gain.
filt = np.sinc(2 * fc * (n - (N - 1) / 2))
w = 0.42 - 0.5 * np.cos(2 * np.pi * n / (N - 1)) + \
    0.08 * np.cos(4 * np.pi * n / (N - 1))
filt *= w
filt /= np.sum(filt)

# Smooth
sflux = np.convolve(p_obs.flux, filt, mode='same')

p_obs = Spectrum1D(spectral_axis=p_obs.wavelength,
                   flux=sflux,
                   uncertainty=p_obs.uncertainty)

In [ ]:
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(p_obs.wavelength, p_obs.flux, linewidth=0.5, label='obs')
pl.plot(p_template.wavelength, p_template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('After smoothing')

## Cross correlate

In [ ]:
# Correlation. 
#
# With no additional specifications, both the entire template and entire spectrum 
# will be included in the correlation computation. This in general will incur in 
# a significant increase in execution time. It is advised that the template is cut
# to work only on the useful region.

corr, lag = correlation.template_correlate(p_obs, p_template)

In [ ]:
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.plot(lag, corr, linewidth=0.5)
pl.xlim(0,300000)
pl.xlabel(lag.unit)

In [ ]:
# Redshift based on maximum
index_peak = np.where(corr == np.amax(corr))[0][0]
v = lag[index_peak]
z = v / const.c.to('km/s')
print("Peak maximum at: ", v)
print("Redshift from peak maximum: ", z)

## Fit correlation peak

In [ ]:
# Redshift based on parabolic fit to mazimum

n = 8 # points to the left or right of correlation maximum

peak_lags = lag[index_peak-n:index_peak+n+1].value
peak_vals = corr[index_peak-n:index_peak+n+1].value
p = np.polyfit(peak_lags, peak_vals, deg=2)
roots = np.roots(p)

v_fit = np.mean(roots) * u.km/u.s # maximum lies at mid point between roots
z = v_fit / const.c.to('km/s')

print("Parabolic fit with maximum at: ", v_fit)
print("Redshift from parabolic fit: ", z)

## Visual check

In [ ]:
pl.figure()
pl.gcf().set_size_inches((8.,4.))
# pl.xlim(sp_xlim)
pl.scatter(peak_lags, peak_vals, label='data')
pl.plot(peak_lags, np.polyval(p, peak_lags), linewidth=0.5, label='fit')
pl.xlabel(lag.unit)
pl.legend()
pl.title('Fit to correlation peak')

In [ ]:
z_ref = 0.758 # "true" redshift, corresponding to 227242.6 km/s

template_z = Spectrum1D(spectral_axis=template.wavelength * (1.+z), flux=template.flux)

In [ ]:
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(obs.wavelength, obs.flux, linewidth=0.5, label='obs')
pl.plot(template_z.wavelength, template_z.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('Redshifted original template and original observed spectrum')

In [ ]:
z_err = (z - z_ref) / z_ref * 100.
print("Error in the derived redshift: ", z_err, "%")

## Case with lower resolution observed spectrum

### Read observation and template:

In [ ]:
# Observation
hdu1d = fits.open(file1d)
hdu1dwht = fits.open(file1dwht)

flux = hdu1d[0].data
wht = hdu1dwht[0].data
unc = 1./ np.sqrt(wht)
wave = np.arange(flux.shape[0])*hdu1d[0].header['CD1_1'] + hdu1d[0].header['CRVAL1']

spec_unit = u.Unit('10^-19 erg s^-1 cm^-2 angstrom^-1')
dataspec = QTable([wave*u.angstrom, flux*spec_unit, wht, unc*spec_unit], 
                   names=('wavelength','flux','weight','uncertainty'))
dataspec_sub = dataspec[dataspec['weight']>0.]
dataspec_sub

In [ ]:
# Now make it into a Spectrum1D instance.
obs_orig = Spectrum1D(spectral_axis=dataspec_sub['wavelength'], 
                 flux=dataspec_sub['flux'], 
                 uncertainty=StdDevUncertainty(dataspec_sub['uncertainty']))

In [ ]:
# Change resolution of spectrum
obs = gaussian_smooth(obs_orig, stddev=15)

In [ ]:
# Template
template = ascii.read(template_file)
factor = 2.E-5 * obs.flux.unit # normalize template to a sensible range
template = Spectrum1D(spectral_axis=template['col1']*u.AA, 
                      flux=template['col2']*factor)

In [ ]:
# Note that in this and in subsequent plots, we are showing just the wavelength range of
# interest. The template covers a significantly wider range.
pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(obs.wavelength, obs.flux, linewidth=0.5, label='obs')
pl.plot(template.wavelength, template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('Input data')

## Preprocess Spectra


### Subtract continuum

In [ ]:
continuum_model = continuum.fit_generic_continuum(obs) 
p_obs = obs - continuum_model(obs.wavelength)
continuum_model = continuum.fit_generic_continuum(template, model=Chebyshev1D(5)) 
p_template = template - continuum_model(template.wavelength)

pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(p_obs.wavelength, p_obs.flux, linewidth=0.5, label='obs')
pl.plot(p_template.wavelength, p_template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('After continuum subtraction')

### Smooth observation


In [ ]:
# Smooth data with sinc kernel
fc = 0.25  # Cutoff frequency as a fraction of the sampling rate (in (0, 0.5)).
b = 0.49   # Transition band, as a fraction of the sampling rate (in (0, 0.5)).

# The IRAF task uses the above values. Here, we try
# a much lower cutoff frequency to really dampen the
# high frequency structure in the observed spectrum.
fc = 0.05 

N = int(np.ceil((4 / b)))
if not N % 2:  # N must be odd
    N += 1
n = np.arange(N)
 
# Compute sinc filter and Blackman window. Multiply filter 
# by window and normalize to get unity gain.
filt = np.sinc(2 * fc * (n - (N - 1) / 2))
w = 0.42 - 0.5 * np.cos(2 * np.pi * n / (N - 1)) + \
    0.08 * np.cos(4 * np.pi * n / (N - 1))
filt *= w
filt /= np.sum(filt)

# Smooth
sflux = np.convolve(p_obs.flux, filt, mode='same')

p_obs = Spectrum1D(spectral_axis=p_obs.wavelength,
                   flux=sflux,
                   uncertainty=p_obs.uncertainty)

pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(p_obs.wavelength, p_obs.flux, linewidth=0.5, label='obs')
pl.plot(p_template.wavelength, p_template.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('After smoothing')

## Cross correlate

In [ ]:
corr, lag = correlation.template_correlate(p_obs, p_template)

pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.plot(lag, corr, linewidth=0.5)
pl.xlim(0,300000)
pl.xlabel(lag.unit)

In [ ]:
# Redshift based on maximum
index_peak = np.where(corr == np.amax(corr))[0][0]
v = lag[index_peak]
z = v / const.c.to('km/s')
print("Peak maximum at: ", v)
print("Redshift from peak maximum: ", z)

# Redshift based on parabolic fit to mazimum

n = 8 # points to the left or right of correlation maximum

peak_lags = lag[index_peak-n:index_peak+n+1].value
peak_vals = corr[index_peak-n:index_peak+n+1].value
p = np.polyfit(peak_lags, peak_vals, deg=2)
roots = np.roots(p)

v_fit = np.mean(roots) * u.km/u.s # maximum lies at mid point between roots
z = v_fit / const.c.to('km/s')

print("")
print("Parabolic fit with maximum at: ", v_fit)
print("Redshift from parabolic fit: ", z)

In [ ]:
z_ref = 0.758 # "true" redshift, corresponding to 227242.6 km/s

template_z = Spectrum1D(spectral_axis=template.wavelength * (1.+z), flux=template.flux)

pl.figure()
pl.gcf().set_size_inches((8.,4.))
pl.xlim(sp_xlim)
pl.plot(obs.wavelength, obs.flux, linewidth=0.5, label='obs')
pl.plot(template_z.wavelength, template_z.flux, linewidth=0.5, color='r', label='template')
pl.legend()
pl.title('Redshifted original template and original observed spectrum')

In [ ]:
z_err = (z - z_ref) / z_ref * 100.
print("Error in the derived redshift: ", z_err, "%")